In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
#Import Censored Data
censored_df=pd.read_json('../ExtractedData/input_withheldtweets.json', lines=True)

In [3]:
#import mostly not censored data
uncensored_df=pd.read_json('../Data/Mostly_Not_Censored.json', lines=True)

In [4]:
#merge and shufflem
df = pd.concat([censored_df, uncensored_df], ignore_index=True, sort=False)
df = shuffle(df)

In [5]:
print("censored", df["withheld_in_countries"].notnull().sum())
print("uncensored", df["withheld_in_countries"].isnull().sum())

censored 2062
uncensored 2381


In [6]:
df.loc[df["withheld_in_countries"].notnull(),"withheld_in_countries",]=0
df.loc[df["withheld_in_countries"].isnull(),"withheld_in_countries",]=1

In [7]:
df_x=df["text"]
df_y=df["withheld_in_countries"]

In [8]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=4)

In [9]:
cv = TfidfVectorizer(min_df = 1, stop_words='english', lowercase='True')

x_traincv=cv.fit_transform(x_train)
x_testcv=cv.transform(x_test)
x_traincv.shape

(3554, 14654)

In [10]:
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [11]:
log_reg = LogisticRegression()

In [12]:
log_reg.fit(x_traincv, y_train)

LogisticRegression()

In [13]:
predictions=log_reg.predict(x_testcv)

In [14]:
confusion_matrix(y_test, predictions)

array([[343,  61],
       [ 60, 425]], dtype=int64)

In [15]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85       404
           1       0.87      0.88      0.88       485

    accuracy                           0.86       889
   macro avg       0.86      0.86      0.86       889
weighted avg       0.86      0.86      0.86       889



In [16]:
log_reg.score(x_testcv, y_test)

0.8638920134983127